In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re


'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

#視窗大小設定 (寬，高)
driver.set_window_size(1200, 960)

# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化


'''
自訂變數
'''
# 放置爬取的資料
listData = []

# 放置 ig 每個格子裡面的超連結
listLink = []

# 分析時暫存用的 dict
dictTmp = {}


'''
以 function 名稱，作為爬蟲流程
'''
def login():
    # 前往首頁
    driver.get('https://www.instagram.com');
    
    # 等待互動元素出現 (這裡用帳號文字欄位)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located( 
            (By.CSS_SELECTOR, 'input[name="username"]') 
        )
    )
    
    # 輸入帳號
    driver.find_element(By.CSS_SELECTOR, 'input[name="username"]').send_keys('')
    
    # 輸入密碼
    driver.find_element(By.CSS_SELECTOR, 'input[name="password"]').send_keys('')
    
    # 強制等待
    sleep(5)
    
    # 按下登入
    driver.find_element(By.CSS_SELECTOR, 'button[type="submit"].sqdOP.L3NKy.y3zKF').click()
    
    # 強制等待
    sleep(5)
    
    '''
    若有提示是否要儲存登入資料或是其它要求，選擇「稍後再說」
    
    備註：
        由於 find_elements 回傳 list 格式，
        所以可以用 len() 來取得元素長度，
        判斷元素是否存在
    '''
    # 按下「稍後再說」關閉提示
    if len(driver.find_elements(By.CSS_SELECTOR, 'div.cmbtv button.sqdOP.yWX7d.y3zKF') ) > 0:
        driver.find_element(By.CSS_SELECTOR, 'div.cmbtv button.sqdOP.yWX7d.y3zKF').click()
    
    # 強制等待
    sleep(3)
    
    # 按下「稍後再說」關閉提示
    if len(driver.find_elements(By.CSS_SELECTOR, 'div.mt3GC button.aOOlW.HoLwm') ) > 0:
        driver.find_element(By.CSS_SELECTOR, 'div.mt3GC button.aOOlW.HoLwm').click()

# 走訪頁面
def visit():
    # 前往指定連結
    driver.get('https://www.instagram.com/english.ig_/?hl=zh-tw');

# 滾動頁面
def scroll():
    # 等待篩選元素出現
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located( 
            (By.CSS_SELECTOR, "div.Nnq7C.weEfm") 
        )
    )
    
    # 瀏覽器內部的高度
    innerHeightOfWindow = 0
    
    # 當前捲動的量(高度)
    totalOffset = 0
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while totalOffset <= innerHeightOfWindow:
        # 每次移動高度
        totalOffset += 300;
        
        # 捲動的 js code
        js_scroll = '''(
            function (){{
                window.scrollTo({{
                    top:{}, 
                    behavior: 'smooth' 
                }});
            }})();'''.format(totalOffset)
        
        # 執行 js code
        driver.execute_script(js_scroll)
        
        # 強制等待
        sleep(1)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeightOfWindow = driver.execute_script('return window.document.documentElement.scrollHeight;');
        
        # 強制等待
        sleep(1)
        
        # 印出捲動距離
        print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))
        
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if totalOffset >= 300:
            break

# 取得每個項目的 url
def getUrl():
    # 取得主要元素的集合
    divs = driver.find_elements(By.CSS_SELECTOR, 'div.Nnq7C.weEfm div.v1Nh3.kIKUG._bz0w')
    
    # 逐一檢視元素
    for index, div in enumerate(divs):
        # 測試功能
        if index >= 2:
            break
        
        # 取得圖片連結
        a = div.find_element(By.CSS_SELECTOR, "div.v1Nh3.kIKUG._bz0w a")
        aLink = a.get_attribute('href')
        print("取得網址: {}".format(aLink))
        
        # 放資料到 list 中
        listLink.append(aLink)

# 逐個網頁連結內容進行分析
def parse():
    '''
    告訴函式 dictTmp 是全域變數，
    在函式內部修改 dictTmp，
    也會同時變更在其它區域使用時的內容
    '''
    global dictTmp
    
    for aLink in listLink:
        # 走訪頁面
        driver.get(aLink)

        # 取得 ig 連結的 id
        regex = r'\/p\/([a-zA-Z0-9-_]+)\/'
        pageId = re.search(regex, aLink)[1]
        
        print("網頁 ID: {}".format(pageId))
        
        sleep(2)
        
        # 初始化分析時暫存用的 dict
        dictTmp = {}
        
        '''
        判斷是否有「向右」按鈕，
        若有，則代表會有多個 li；
        若無，則代表只有一個 li
        '''
        if len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
            # 取得多元素資訊
            _parseMultipleItems()
            
            # 整合此次網頁連結的元素資掀
            listData.append({
                "id": pageId,
                "url": aLink,
                "content": dictTmp
            })
            
        else:
            
            if len( driver.find_elements(By.CSS_SELECTOR, "img.FFVAD") ) > 0: # 若是 img
                # 取得 img
                imgSrc = driver.find_element(By.CSS_SELECTOR, "img.FFVAD").get_attribute('src')

                # 雜湊 video 連結，作為 dict 的 key
                strKey = _md5(imgSrc)
                
                # 建立 img 的 key-value
                dictTmp[strKey] = imgSrc

                # 新增元素資訊到全域 list
                listData.append({
                    "id": pageId,
                    "url": aLink,
                    "content": dictTmp
                })
                
            elif len( driver.find_elements(By.CSS_SELECTOR, "video.tWeCl") ) > 0: # 若是 video
                # 取得 video 連結
                videoSrc = driver.find_element(By.CSS_SELECTOR, "video.tWeCl").get_attribute('src')
                
                # 雜湊 video 連結，作為 dict 的 key
                strKey = _md5(videoSrc)
                
                # 建立 video 的 key-value
                dictTmp[strKey] = videoSrc
                
                # 新增元素資訊到全域 list
                listData.append({
                    "id": pageId,
                    "url": aLink,
                    "content": dictTmp
                })

# 取得多元素資訊
def _parseMultipleItems():
    '''
    告訴函式 dictTmp 是全域變數，
    在函式內部修改 dictTmp，
    也會同時變更在其它區域使用時的內容
    '''
    global dictTmp
    
    # 若是有「向右」按鈕，代表還有 li 需要往下按
    if len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
        # 按下向右按鈕
        driver.find_element(By.CSS_SELECTOR, "button._6CZji").click()
        
        '''
        結合雜湊功能，建立 dict 的 key，
        再將元素裡面的屬性值，視為 value 整合在 dict 當中
        '''
        elements = driver.find_elements(By.CSS_SELECTOR, "li.Ckrof")
        
        # 檢視各個 li
        for li in elements:
            
            if len(li.find_elements(By.CSS_SELECTOR, "img.FFVAD")) > 0: # 如果這個 li 裡面有 img
                # 取得 img 連結
                imgSrc = li.find_element(By.CSS_SELECTOR, "img.FFVAD").get_attribute('src')
                
                # 雜湊 img 連結，作為 dict 的 key
                strKey = _md5(imgSrc)
                
                # 建立 img 的 key-value
                dictTmp[strKey] = imgSrc
                
            elif len(li.find_elements(By.CSS_SELECTOR, "video.tWeCl")) > 0: # 如果這個 li 裡面有 video
                # 取得 video 連結
                videoSrc = li.find_element(By.CSS_SELECTOR, "video.tWeCl").get_attribute('src')
                
                # 雜湊 video 連結，作為 dict 的 key
                strKey = _md5(videoSrc)
                
                # 建立 video 的 key-value
                dictTmp[strKey] = videoSrc
        
        # 強制等待
        sleep(2)
        
        # 呼叫自己，繼續按「向右」按鈕，直到沒有「向右」按鈕，才結束
        _parseMultipleItems()

# 建立雜湊值
def _md5(string):
    '''
    說明：
    建立雜湊機制

    用法：
    m = hashlib.md5()              # 建立 MD5 物件
    str = 'yourValue'              # 建立想要雜湊的值
    m.update(str.encode("utf-8"))  # 更新 MD5 雜湊值
    strHash = m.hexdigest()        # 取得 MD5 雜湊值
    '''
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    strKey = m.hexdigest()
    return strKey 
        
# 將 list 存成 json
def saveJson():
    fp = open("ig.json", "w", encoding='utf-8')
    fp.write( json.dumps(listData, ensure_ascii=False) )
    fp.close()
    
# 關閉瀏覽器
def close():
    driver.quit()

# ??????
def download():
    # 開啟 json 檔案
    fp = open("ig.json", "r", encoding='utf-8')
    
    #取得 json 字串
    strJson = fp.read()
    
    # 關閉檔案
    fp.close()
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    
    # ??????
    for obj in listData:
        for key in obj['content']:
            print("下載連結: {}".format( obj['content'][key] ))
            dl_link = re.search(r'https?:\/\/\S+.\/(\S+\.(jpe?g|mp4))', obj['content'][key])[1]
            os.system('curl -o {} "{}"'.format(dl_link, obj['content'][key]))
            
'''主程式'''
login()
visit()
scroll()
getUrl()
parse()
saveJson()
close()
download()